# I55 : Use repr Strings for Debugging Output

- When debuggin a Python program, the print function (or output via the logging built-in module) will get you surprisingly far. Python internals are often easy to access via plain attriutes. All you need to do is print how the state of your program changes while it runs and see where it goes wrong.

- The print function outputs a human-readable string version of whatever you supply it. For example, priting a basic string will print the contents of the string without the surrounding quote characters.

In [1]:
print('foo bar')

foo bar


- This is equivalent to using the '%s' format string and the % operator.

In [2]:
print('%s' % 'foo bar')

foo bar


- The problem is that the human -readable string for a value doesn't make it clear what the actual type of the value is. For example, notice how in the default output of print you can't distinguish between the types of the number 5 and the string '5'.

In [3]:
print(5)
print('5')

5
5


- If you're debuggin a program with print, these type differences matter. What you almost always want while debugging is to see the repr version of an object. The repr built-in function returns the *printable representation* of an object, which should be its most clearly understandable string representation. For built-in types, the string returned by repr is a valid Python expression.

In [4]:
a = '\x07'
print(repr(a))

'\x07'


- Passing the value from repr to the eval built-in function should result in the same Python object you started with(of course, in practice, you should only use eval with extreme caution)

In [5]:
b = eval(repr(a))
assert a == b

- When you're debugging with print, you should repr the value before printing to ensure that any difference in types is clear.

In [6]:
print(repr(5))
print(repr('5'))

5
'5'


- This is equivalent to using the '%r' format string and the % operator.

In [7]:
print('%r' % 5)
print('%r' % '5')

5
'5'


- For dynamic Python objects, the default human-readable string value is the same as teh repr value. This means that passing a dynamic object to print will do the right thing, and you don't need to explicitly call repr on it. Unfortunately, the default value of repr for object instances isn't especially helpful. For example, here I define a simple class and then print its value:

In [8]:
class OpaqueClass(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
obj = OpaqueClass(1, 2)
print(obj)

- This output can't be passed to the eval function, and it says nothing about the instance fields of the object. 
- There are two solutions to this problem. If you have control of the class, you can define your own \_\_repr\_\_ special method that returns a string containing the Python expression that recreates the object. Here, I define that function for the class above:

In [10]:
class BetterClass(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return 'BetterClass(%d, %d)' % (self.x, self.y)
    
obj = BetterClass(1, 2)
print(obj)

BetterClass(1, 2)


- When you don't have control over the class definition, you can reach into the object's instance dictionary, which is stored in the \_\_dict\_\_ attribute. Here, I print out the contents of an OpaqueClass instance:

In [11]:
obj = OpaqueClass(4, 5)
print(obj.__dict__)

{'x': 4, 'y': 5}


## Things to Remember
- Calling print on built-in Python types will produce the human-readable string version of a value, which hides type information.
- Callling repr on built-in Python types will produce the printable string version of a value. These repr strings could be passed to the eval built-in function to get back the original value.
- %s in format strings will produce human-readable strings like str.%r will produce printable strings like repr.
- You can define the \_\_repr\_\_ method to customize the printable representation of a class and provide more detailed debugging information.
- You can reach into any objecy's \_\_dict\_\_ attribute to view its internals.

# I56 : Test Everything with unittest 

- Python doesn't have static type checking. There's nothings in the compiler that will ensure that your program will work when you run it. With Python you don't know whether the functions your programs calls will be defined at runtime, even when their existence is evident in the source code. This dynamic behavior is a blessing and a curse.

- The large numbers of Python programmers out there say it's worth it because of the productivity gained from the resulting brevity and simplicity. But more people have heard at least one horror story about Python in which a program encounted a boneheaded error at runtime.

- One of the worst examples I've heard is when SyntaxError was raised in production as a side effect of a dynamic import. The programmer I know who was hit by this surprising occurrence has since ruled out using Python ever again.

- But I have to wonder, why wasn't the code tested before the program was deployed to production? Type safety isn't everything. You should always test your code, regardless of what language it's written in. However, I'll admit that the big difference between Python and many other languages is that the only way to have *any* confidence in Python program is by writing tests. There is no veil of static type checking to make you feel safe.

- Luckily, the same dynamic features that prevent static type checking in Python also make it extremely easy to write tests for your code. You can use Python's dynamic nature and easily overridable behaviors to implement tests and ensure that your programs work as expected.

- You should think of tests as ana insurance policy on your code. Good tests give you confidence that your code is correct. If you refactor or expand your code, tests make it easy to identify how behaviors have changed. It sounds counter-intuitive, but having good tests actually makes it easier to modify Python code, not harder.

- The simplest way to write tests is to use the unittest built-in module. For example, say you have the following utility function defined in utils.py:

In [15]:
def to_str(data):
    if isinstance(data, str):
        return data
    elif isinstance(data, bytes):
        return data.decode('utf-8')
    else:
        raise TypeError('Must supply str or bytes, '
                        'found: %r' % data)

- To define tests, I create a second file named test\_utils.py or utils\_test.py that contains tests for each behavior I expect.

In [16]:
from unittest import TestCase, main
# from utils import to_str

class UtilsTestCase(TestCase):
    def test_to_str_bytes(self):
        self.assertEqual('hello', to_str(b'hello'))
        
    def test_to_str_str(self):
        self.assertEqual('hello', to_str('hello'))
        
    def test_to_str_bad(self):
        self.assertRaises(TypeError, to_str, object())
        
if __name__ == '__main__':
    main()

E
ERROR: /home/dockeruser/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/home/dockeruser/'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/home/dockeruser/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


- Tests are organized into TestCase classes. Each test is a method beginning with the word test. If a test method runs without raising any kind of Exception (including AssertionError from assert statements), then the test is considered to have passed successfully.

- The TestCase class provides helper mehtods for making assertions in your test, such as assertEqual for verifying eqaulity, assertTrue for verifying Boolean expressions, and assertRaises for verifying that exceptions are raised when appropriate. You can define your own helper methods in TestCase subclasses to make your tests more readable; just ensure that your method names don't begin with the word test.

- Sometimes, your TestCase classes need to set up the test environment before running test methods. To do this, you can override the setUp and tearDown methods. These methods are called before and after each test method, respectively, and they let you ensure that each test runs in isolation. For example, here I define a TestCase that creates a temporary directory before each test and deletes its contents after each test finishes:

In [17]:
class MyTest(TestCase):
    def setUp(self):
        self.test_dir = TemporaryDirectory()
    
    def tearDown(self):
        self.test_dir.cleanup()
        
    pass

- I usually define one TestCase for each set of related tests. Sometimes I have one TestCase for each function taht has many edge cases. Other times, a TestCase spans all functions in a single module. I'll also create one TestCase for testing a single class and all of its methods.

- When programs get complicated, you'll want additional tests for verifying the interactions between your modules, instead of only testing code in isolation. This is the difference between *unit tests* and *integration tests*. In Python, it's important to write both types of tests for exactly the same reason: You have no guarantee that your modules will actually work together unless you prove it.

## Things to Remeber 
- The only way to have confidence in Python program is to write tests.
- The unittest built-in module provides most of the facilities you'll need to write good tests.
- You can define tests by subclassing TestCase and defineing one method per behavior you'd like to test. Test methods on TestCase classes must start with the word test.

- It's important to write both unit tests and integration tests